In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import healpy as hp

from pyoperators import MPI

from qubic.lib.Qdictionary import qubicDict
from qubic.lib.Calibration.Qselfcal import scatter_plot_FP, get_TEScoordinates_ONAFP
from qubic.lib.Instrument.Qinstrument import QubicInstrument
from qubic.lib.Instrument.Qacquisition import QubicMultiAcquisitions
from qubic.lib.Qsamplings import get_pointing, equ2gal

In [ ]:
nside = 128

# Initialize QUBIC instance

Build the QUBIC dictionary, mandatory to use the QUBIC functions

In [ ]:
def get_dict(comm, key='in'):
    """QUBIC dictionary.

    Method to modify the qubic dictionary.

    Parameters
    ----------
    key : str, optional
        Can be "in" or "out".
        It is used to build respectively the instances to generate the TODs or to reconstruct the sky maps,
        by default "in".

    Returns
    -------
    dict_qubic: dict
        Modified QUBIC dictionary.

    """

    args = {
        "npointings": 1,
        "nf_recon": 2,
        "nf_sub": 10,
        "nside": nside,
        "MultiBand": True,
        "period": 1,
        "RA_center": 0,
        "DEC_center": -57,
        "filter_nu": 150 * 1e9,
        "noiseless": True,
        "comm": comm,
        "dtheta": 15,
        "nprocs_sampling": 1,
        "nprocs_instrument": comm.Get_size(),
        "photon_noise": False,
        "nhwp_angles": 1,
        "effective_duration150": 3,
        "effective_duration220": 3,
        "filter_relative_bandwidth": 0.25,
        "type_instrument": "two",
        "TemperatureAtmosphere150": None,
        "TemperatureAtmosphere220": None,
        "EmissivityAtmosphere150": None,
        "EmissivityAtmosphere220": None,
        "detector_nep": float(4.7e-17),
        "synthbeam_kmax": 1,
        "synthbeam_fraction": 1,
        "beam_shape": "gaussian",
        "debug":False,
        "kind": "IQU",
    }

    ### Get the default dictionary
    dictfilename = "dicts/pipeline_demo.dict"
    dict_qubic = qubicDict()
    dict_qubic.read_from_file(dictfilename)

    for i in args.keys():

        dict_qubic[str(i)] = args[i]

    return dict_qubic

In [ ]:
# Build MPI communicator and qubic dict
comm = MPI.COMM_WORLD
dict_qubic = get_dict(comm)

In [ ]:
# dict_qubic

In [ ]:
dict_qubic["use_synthbeam_fits_file"] = False

In [ ]:
# Build scanning strategy
sampling = get_pointing(dict_qubic)

# Build QubicInstrument
q = QubicInstrument(dict_qubic)

In [ ]:
# Number of sub-acquisitions
nsub_max = 70
nsub_list = np.arange(10, nsub_max + 1, 10)

In [ ]:
TOD_list = []
freq_list = []
H_list = []

for insub in nsub_list:
    # Build input uniform maps
    input_maps = np.ones((insub, hp.nside2npix(nside), 3))
    
    # Build acquisition operator
    Qacq = QubicMultiAcquisitions(dict_qubic, 2*insub, 2, sampling=sampling)
    H = Qacq.H[:int(insub/2)]
    
    # Build TOD
    tod = []
    for i in range(int(insub/2)):
        tod.append(H[i](input_maps[i]))

    # Store everything
    H_list.append(H)
    TOD_list.append(np.array(tod))
    freq_list.append(np.array(Qacq.allnus))

In [ ]:
np.shape(TOD_list[0])

In [ ]:
### Sum detectors

for i in range(len(nsub_list)):
    plt.plot(nsub_list[i], np.sum(np.mean(TOD_list[i], axis=0), axis=0)[0], '.r')

plt.xlabel('Nsub')
plt.ylabel('TOD')
plt.title("Sum over detectors")

In [ ]:
TOD_list[0].shape

In [ ]:
### For one detector
idet = 0

for i in range(len(nsub_list)):
    plt.plot(nsub_list[i], np.mean(TOD_list[i], axis=0)[0], '.r')

plt.xlabel('Nsub')
plt.ylabel('TOD')

In [ ]:
### For all detectors 

for i in range(len(nsub_list)):
    for idet in range(992):
        plt.plot(nsub_list[i], np.mean(TOD_list[i], axis=0)[idet], '.')

plt.xlabel('Nsub')
plt.ylabel('TOD')
plt.title('Convergence of TOD for all detectors')